In [ ]:
import torch
import torchvision

#Criando um transformador de imgs em tensores.
#torch só lida com tensores(forma númerica de representar informações)
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

#baixar o dataset, sem download e diferença no train
# trainset = torchvision.datasets.LSUN(root='./data',  classes='train', transform=transform)
# testset = torchvision.datasets.LSUN(root='./data', classes='test', transform=transform)

#70mil
trainset = torchvision.datasets.MNIST(root='./data',train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

#divisão, serem processados juntos
#testar 32, 64, 128 e 256
batch_size = 32

#dividir o dataset
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 89417246.63it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 57301652.71it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 24393043.84it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 9640955.85it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

#erro no calculo de entrada e saida
# class MNISTModel(nn.Module):
#       def __init__(self):
#         super().__init__()

#         self.conv1 = nn.Conv2d(1, 28, kernel_size=(3,3), stride=1, padding=1)
#         self.act1 = nn.ReLU()
#         self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))

#         self.conv2 = nn.Conv2d(28, 28, kernel_size=(3,3), stride=1, padding=1)
#         self.act2 = nn.ReLU()
#         self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))

#         self.flat = nn.Flatten()

#         self.fc3 = nn.Linear(56*28, 28)
#         self.act3 = nn.ReLU()
#         self.drop3 = nn.Dropout(0.5)
#         self.fc4 = nn.Linear(28, 10)

#       def forward(self, x):
#         x = self.act1(self.conv1(x))
#         x = self.act2(self.conv2(x))
#         x = self.pool2(x)
#         x = self.flat(x)
#         x = self.act3(self.fc3(x))
#         x = self.drop3(x)
#         x = self.fc4(x)
#         return x

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        #camada conectada, camada linear
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        #primeira camada convolucional seguida de uma função de ativação ReLU e uma operação de max pooling
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        #Redimensionar o tensor de saída, -1: automaticamente
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        #aplica a função de ativação log-softmax à saída da segunda camada totalmente conectada.
        #paça a chance de ser a classe
        return F.log_softmax(x)



# model = MNISTModel()
model = Net()
loss_fn = nn.CrossEntropyLoss()
#lr maior: divergir
#colocar lr maior no começo e depois diminuir?
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

#hodout
n_epochs = 10
for epoch in range(n_epochs):
    print(f'epoch {epoch}')
    for inputs, labels in trainloader:
        # forward, backward, and then weight update
        y_pred = model(inputs)
        loss = loss_fn(y_pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    acc = 0
    count = 0
    for inputs, labels in testloader:
        y_pred = model(inputs)
        acc += (torch.argmax(y_pred, 1) == labels).float().sum()
        count += len(labels)
    acc /= count
    print("Epoch %d: model accuracy %.2f%%" % (epoch, acc*100))

torch.save(model.state_dict(), "MNISTmodel.pth")

epoch 0


<ipython-input-37-18700019b494>:49: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Epoch 0: model accuracy 90.20%
epoch 1
Epoch 1: model accuracy 92.78%
epoch 2
Epoch 2: model accuracy 94.16%
epoch 3
Epoch 3: model accuracy 94.62%
epoch 4
Epoch 4: model accuracy 95.08%
epoch 5
Epoch 5: model accuracy 95.53%
epoch 6
Epoch 6: model accuracy 95.59%
epoch 7
Epoch 7: model accuracy 95.94%
epoch 8
Epoch 8: model accuracy 95.34%
epoch 9
Epoch 9: model accuracy 96.19%
